# Carla Playground

## Carla set-up

In [1]:
import carla 
import math 
import random 
import time 

In [7]:
import glob
import os
import sys
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
    sys.version_info.major,
    sys.version_info.minor,
    'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

In [8]:
client = carla.Client('localhost', 2000) 
world = client.get_world() 

In [4]:
# settings = world.get_settings()
# settings.synchronous_mode = True # Enables synchronous mode
# # settings.fixed_delta_seconds = 0.05
# world.apply_settings(settings)

## Fix spectator on top of intersection

In [10]:
spectatorLoc = carla.Location(x=-47, y=16.8, z=60)
spectator = world.get_spectator() 
transform = carla.Transform(spectatorLoc, carla.Rotation(-90)) 
spectator.set_transform(transform)

## Get spawn points and label on map

In [13]:
spawn_points = world.get_map().get_spawn_points()

In [92]:
for i, spawn_point in enumerate(spawn_points):
    world.debug.draw_string(spawn_point.location, str(i), life_time=1000)

## Control car by varying throttle manually

In [98]:
bp_lib = world.get_blueprint_library() 
vehicle_bp = bp_lib.find('vehicle.jeep.wrangler_rubicon')
vehicle1 = world.spawn_actor(vehicle_bp, spawn_points[28])

In [96]:
def detect_collision(vehicle, collision_box):
    if  ((vehicle.get_location().x >= collision_box.location.x - 1) and \
        (vehicle.get_location().x <= collision_box.location.x + 1)) and \
        ((vehicle.get_location().y >= collision_box.location.y - 1) and \
        (vehicle.get_location().y <= collision_box.location.y + 1)):
            print('reached')
            print(str(vehicle.get_location().x), ' ', str(vehicle.get_location().y))
            return True
    return False

In [99]:
vehicle1.apply_control(carla.VehicleControl(throttle = 1.0))
counter = 0
while True:
    counter += 1
#     if counter % 1000 == 0:
#         print(str(vehicle1.get_location().x) + ' ' + str(vehicle1.get_location().y))
    if detect_collision(vehicle1, spawn_points[79]):
        vehicle1.apply_control(carla.VehicleControl(hand_brake = True))
        time.sleep(50)
        vehicle1.destroy()
        break

reached
-41.76508331298828   49.77585220336914


## Control car using traffic manager with custom path

In [ ]:
'''
Paths to make car move through the intersection (Direction is relative to the fixed spectator pov)
N->E (49, 27)
N->S (49, 3)
N->W (50, 67)

E->N (28, 57)
E->S (137, 77)
E->W (28, 95)

S->E (85, 26)
S->N (80, 56)
S->W (80, 95)

W->S (138, 2)
W->E (139, 27)
W->N (139, 56)
'''

In [ ]:
bp_lib = world.get_blueprint_library() 
vehicle_bp = bp_lib.find('vehicle.jeep.wrangler_rubicon')
vehicle1 = world.spawn_actor(vehicle_bp, spawn_points[50])

In [17]:
route = [spawn_points[50].location, spawn_points[67].location]

In [18]:
traffic_manager = client.get_trafficmanager(2000)

In [19]:
#Set all traffic lights in the map to green
list_actor = world.get_actors()
for actor_ in list_actor:
    if isinstance(actor_, carla.TrafficLight):
        actor_.set_state(carla.TrafficLightState.Green) 
        actor_.set_green_time(1000.0)

In [20]:
vehicle.set_autopilot(True)

traffic_manager.update_vehicle_lights(vehicle, True)
traffic_manager.random_left_lanechange_percentage(vehicle, 0)
traffic_manager.random_right_lanechange_percentage(vehicle, 0)
traffic_manager.auto_lane_change(vehicle, False)
traffic_manager.ignore_lights_percentage(vehicle, 100)
traffic_manager.ignore_vehicles_percentage(vehicle, 100)
traffic_manager.ignore_signs_percentage(vehicle, 100)

traffic_manager.set_path(vehicle, route)
